# **Desafio Técnico: RenovaBR**

Este projeto implementa técnicas típicas de Ciência de Dados ao desafio proposto pelo **RenovaBR**. O desafio consiste em **analisar dados eleitorais do estado de São Paulo e seus municípios nas eleições de 2020**. Conforme sugerido pelo RenovaBR no [documento que descreve o desafio](../desafio-tecnico.pdf), o projeto segue o seguinte roteiro:

🧹 Pré-processamento e tratamento dos dados

🐘 Dump dos dados tratados em um banco de dados relacional

⌨️ Acesso aos dados via SQL em Python

📊 Análise Exploratória dos Dados

A análise exploratória estará conjugada à resolução de algumas questões propostas, mas não se limitará a elas. O objetivo é explorar os dados e extrair informações relevantes para a tomada de decisão.

---

### **📚 Importando as bibliotecas**

In [1]:
# processamento e analise de dados
import pandas as pd

# visualizacao de dados
import matplotlib.pyplot as plt
import seaborn as sns

# geoprocessamento de dados
import geopandas as gpd
import geobr

# versoes dos pacotes, python e sistema operacional
import session_info
session_info.show(html = False, dependencies = False)

-----
geobr               NA
geopandas           0.7.0
matplotlib          3.7.0
pandas              2.0.2
seaborn             0.12.2
session_info        1.0.0
-----
IPython             8.11.0
jupyter_client      8.0.3
jupyter_core        5.2.0
notebook            6.5.3
-----
Python 3.11.3 (main, Apr  7 2023, 19:29:16) [Clang 14.0.0 (clang-1400.0.29.202)]
macOS-12.6.6-x86_64-i386-64bit
-----
Session information updated at 2023-08-17 08:52


### **🧹 Pré-processamento e tratamento dos dados**

In [2]:
# mude seu caminho ate o arquivo aqui
PATH_TO_ELEITORADO = "../database_desafio_renovaBR/eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv"
PATH_TO_RESULTADOS = "../database_desafio_renovaBR/resultados/SP_turno_1.csv"

# leitura dos arquivos
eleitorado = pd.read_csv(PATH_TO_ELEITORADO, sep=";", encoding="latin1")
resultados = pd.read_csv(PATH_TO_RESULTADOS, sep=";", encoding="latin1")

Agora que lemos os dados e os colocamos em um DataFrame, podemos começar a analisá-los. Primeiro, como sabemos que os arquivos `.csv` são bastante grandes, vamos verificar se há entradas que podem ser removidas logo de cara. Por exemplo, sabemos que estamos interessados nos dados apenas do estado de São Paulo. Caso haja informações sobre outros estados no DataFrame, podemos removê-las. Vejamos:

In [3]:
eleitorado["SG_UF"].value_counts().head()

SG_UF
SP    639502
MG    609026
RS    320563
BA    317883
PR    287673
Name: count, dtype: int64

Vamos filtrar os casos correspondentes apenas ao estado de São Paulo.

In [4]:
# filtrar apenas os eleitores de SP
eleitorado = eleitorado.query("SG_UF == 'SP'")

# check 
eleitorado["SG_UF"].value_counts()

SG_UF
SP    639502
Name: count, dtype: int64

No caso dos dados de resultados eleitorais, a tabela já se restringe ao estado desejado. Portanto, não é necessário filtrar os dados em relação ao estado.

In [5]:
# check
resultados["SG_UF"].value_counts()

SG_UF
SP    9434000
Name: count, dtype: int64

Agora, vamos analisar as colunas do nosso dataset e verificar quais colunas podem ser removidas. Para isso, é importante entender o que cada coluna representa. Isso será possível através do dicionário de dados [dicionário de dados](../database_desafio_renovaBR/eleitorado/perfil_eleitorado_2020/leiame.pdf). Naturalmente, a escolha das colunas a serem mantidas ou removidas é subjetiva e depende do objetivo da análise. Neste caso, **vamos remover as colunas que não serão utilizadas na análise.**

Veja as colunas do dataset `eleitorado`:

In [6]:
# checando as colunas do dataset
eleitorado.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

Para responder às perguntas de negócio, vale a pena mantermos as colunas indicadas abaixo. Note que normalmente não removeríamos as colunas `ANO_ELEICAO` e `SG_UF`, mas, como estamos analisando apenas um estado e um ano, elas não são relevantes para a análise.

In [7]:
# definindo as colunas de interesse
eleitorado_colunas = ["CD_MUNICIPIO", "NM_MUNICIPIO", "NR_ZONA", 
                      "DS_GENERO", "DS_ESTADO_CIVIL", "DS_FAIXA_ETARIA", 
                      "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL"]

# Filtrando as colunas de interesse
eleitorado_filt = eleitorado[eleitorado_colunas]

# Checando se o número de linhas do dataset original é igual ao número de linhas do dataset filtrado
assert eleitorado.shape[0] == eleitorado_filt.drop_duplicates().shape[0]

Agora faremos o mesmo para o dataset `resultados`. Vale o comentário relacionado às colunas `ANO_ELEICAO` e `SG_UF`.

In [8]:
# checando as colunas do dataset
resultados.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_PLEITO', 'DT_PLEITO', 'NR_TURNO', 'CD_ELEICAO',
       'DS_ELEICAO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA',
       'NR_SECAO', 'NR_LOCAL_VOTACAO', 'CD_CARGO_PERGUNTA',
       'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO',
       'DT_BU_RECEBIDO', 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES',
       'CD_TIPO_URNA', 'DS_TIPO_URNA', 'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_URNA_EFETIVADA',
       'CD_CARGA_1_URNA_EFETIVADA', 'CD_CARGA_2_URNA_EFETIVADA',
       'CD_FLASHCARD_URNA_EFETIVADA', 'DT_CARGA_URNA_EFETIVADA',
       'DS_CARGO_PERGUNTA_SECAO', 'DS_AGREGADAS', 'DT_ABERTURA',
       'DT_ENCERRAMENTO', 'QT_ELEITORES_BIOMETRIA_NH', 'DT_EMISSAO_BU',
       'NR_JUNTA_APURADORA', 'NR_TURMA_APURADORA'],
      dtype='object')

In [9]:
# definindo as colunas de interesse
resultados_colunas = ["NM_TIPO_ELEICAO", "DS_ELEICAO", "CD_MUNICIPIO", 
                      "NM_MUNICIPIO", "NR_ZONA", "NR_SECAO",
                      "NR_LOCAL_VOTACAO", "DS_CARGO_PERGUNTA", "SG_PARTIDO",
                      "NR_VOTAVEL", "NM_VOTAVEL", "QT_VOTOS"]

# Filtrando as colunas de interesse
resultados_filt = resultados[resultados_colunas]

# Checando se o número de linhas do dataset original é igual ao número de linhas do dataset filtrado
assert resultados_filt.drop_duplicates().shape[0] == resultados.shape[0]

O próximo passo é verificar quais registros brancos ou nulos podem ser removidos da base. 

Segundo as informações do dicionário, algumas especificações importantes sobre o dataset `eleitorado` são:
- Campos preenchidos como #NULO significam que a informação está em branco;
- Campos preenchidos com #NE significa que a informação não está disponível.

Vamos buscar por esses valores e verificar se há registros a serem removidos.

In [31]:
for columns in eleitorado_filt.columns:
    flt_nulo = eleitorado_filt[columns] == "#NULO"
    flt_ne = eleitorado_filt[columns] == "#NE"
    
    n_nulo = eleitorado_filt[flt_nulo].shape[0]
    n_ne = eleitorado_filt[flt_ne].shape[0]
    
    if n_nulo > 0 or n_ne > 0:
        print(f"Coluna: {columns} | #NULO: {n_nulo} | #NE: {n_ne}")
    else:
        print(f"Coluna: {columns} | OK")

Coluna: CD_MUNICIPIO | OK
Coluna: NM_MUNICIPIO | OK
Coluna: NR_ZONA | OK
Coluna: DS_GENERO | OK
Coluna: DS_ESTADO_CIVIL | OK
Coluna: DS_FAIXA_ETARIA | OK
Coluna: DS_GRAU_ESCOLARIDADE | OK
Coluna: QT_ELEITORES_PERFIL | OK


In [32]:
for columns in resultados_filt.columns:
    flt_nulo = resultados_filt[columns] == "Nulo"
    flt_branco = resultados_filt[columns] == "Branco"
    
    n_nulo = resultados_filt[flt_nulo].shape[0]
    n_branco = resultados_filt[flt_branco].shape[0]
    
    if n_nulo > 0 or n_branco > 0:
        print(f"Coluna: {columns} | Nulo: {n_nulo} | Branco: {n_branco}")
    else:
        print(f"Coluna: {columns} | OK")

Coluna: NM_TIPO_ELEICAO | OK
Coluna: DS_ELEICAO | OK
Coluna: CD_MUNICIPIO | OK
Coluna: NM_MUNICIPIO | OK
Coluna: NR_ZONA | OK
Coluna: NR_SECAO | OK
Coluna: NR_LOCAL_VOTACAO | OK
Coluna: DS_CARGO_PERGUNTA | OK
Coluna: SG_PARTIDO | OK
Coluna: NR_VOTAVEL | OK
Coluna: NM_VOTAVEL | Nulo: 171359 | Branco: 171203
Coluna: QT_VOTOS | OK


Essas ocorrências de `NM_VOTAVEL` nulo e branco são justamente devido aos votos nulos e brancos. Vamos remover essas linhas do nosso dataset.

In [41]:
# removendo votos brancos
flt_branco = resultados_filt["NM_VOTAVEL"] == "Branco"
print(f"Foram removidos {resultados_filt[flt_branco]['QT_VOTOS'].sum()} votos brancos.")
resultados_filt = resultados_filt[~flt_branco]

# removendo votos nulos
flt_nulo = resultados_filt["NM_VOTAVEL"] == "Nulo"
print(f"Foram removidos {resultados_filt[flt_nulo]['QT_VOTOS'].sum()} votos nulos.")
resultados_filt = resultados_filt[~flt_nulo]

Foram removidos 2856552 votos brancos


### **🐘 Dump dos dados tratados em um banco de dados relacional**

### **⌨️ Acesso aos dados via SQL em Python**

### **📊 Análise Exploratória dos Dados**

### **Conclusões 💬**